# Extracting constants from expressions and packing them into arrays

In many cases, the expressions in OpenFD are filled with constants that are either symbolic, or of floating-point nature. The symbolic constants might be scalar parameters that needs to be passed to kernel functions. The floating-point values typically appear when operators act on grid functions. In the compute kernels, it is undesirable to have these constants floating around as they can either lead to extra redundant computational work or lead to poor GPU kernel design. In this notebook, we explore how to manipulate expressions by replacing constants with arrays. 

In [1]:
from openfd.dev import manipulators, Constant
from openfd import GridFunction, GridFunctionExpression as Expr
from sympy import symbols

## What is a constant?
By default, constants are the Python datatypes `int`, `float`, and also the sympy types `Int` and `Float`. In addition to these constants OpenFD also supports Symbolic constants `Constant`. To check if something is a constant, we use the following function.

In [2]:
manipulators.isconstant(0.2)

True

In [3]:
manipulators.isconstant(1 + 0.2)

True

This function is able to tell if an expression is constant.

In [4]:
c = Constant('c')
manipulators.isconstant(c + 0.2)

True

In [5]:
c = Constant('c')
manipulators.isconstant(c**2 + 0.2)

True

Grid Functions, and Operators are not considered constants.

In [6]:
u = GridFunction('u', shape=(10,))
manipulators.isconstant(u+c)

False

## Finding constants
While the previous expression is not considered a constant, it certainly contains one! This constant can be identified by using another function `constants` that outputs all list of all unique constants found in an expression.

In [7]:
manipulators.constants(u+c)

[c]

In [8]:
manipulators.constants(c**2*u + c)

[c**2, c]

Some expressions may require some massaging beforehand.

In [9]:
out = manipulators.constants(u*(c*u + c)*c)
out

[c]

We can identify the constant $c^2$ from the previous expression by using Sympy's `expand` function.

In [10]:
from sympy import expand
manipulators.constants(expand(u*(c*u + c)*c))

[c**2]

## Packing constants into arrays

So far, no modifications have been made to the expressions themselves. In this section, the constants will be replaced with arrays. The function that accomplishes this task is called `pack` one of its use cases is for building expressions that later will be executed on GPUs.

In [11]:
expr = Expr(u*(c*u + c)*c)
newexpr, constants = manipulators.pack(expand(expr))

In [12]:
newexpr

d[0]*u + d[0]*u**2

In [13]:
constants

[c**2]

In most cases, we need to evaluate the expression somewhere before packing its constants


In [14]:
newexpr, constants = manipulators.pack(expand(expr[0]))

Sometimes multi-dimensional arrays need to be used for packing data. In this case, we need to provide an `index` function to `pack` that tells it how the output array (`d`) should be accessed. 

In [15]:
j = symbols('j')
index = lambda i : (i, j)
expr = Expr(3*c*u + c**2)
newexpr, constants = manipulators.pack(expand(expr[0]), index=index)

In [16]:
newexpr

d[0, j]*u[0] + d[1, j]

In [17]:
constants

[3*c, c**2]

The `pack` function can return a `dict` instead of a `list` of replaced values.

In [18]:
newexpr, constants = manipulators.pack(expand(expr[0]), index=index, dict=True)

In [19]:
constants

{d[1, j]: c**2, d[0, j]: 3*c}